In [1]:
%pylab --no-import-all

from os import path
import pandas as pd

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
file = path.join("..", "data", "processed", "df.csv")
df = pd.read_csv(file, index_col=0)

In [3]:
no_calc = 0
diff_calc = 0

for idx, row in df.iterrows():
    computed_L_PREOVULATION = None
    for i in range(1, 91):
        six_days = [row['TEMP'+str(i+j)] for j in range(0, 6)]
        three_days = [row['TEMP'+str(i+k)] for k in range(6, 9)]
        if min(three_days) > max(six_days):
            computed_L_PREOVULATION = i + 5
            break
    if computed_L_PREOVULATION is None:
        no_calc += 1
    elif computed_L_PREOVULATION != int(row.L_PREOVULATION):
        diff_calc += 1
total_errors = no_calc + diff_calc
print("Total number of cycles: {}".format(len(df)))
print("Total diffs: {} ({:5.2f}%)".format(total_errors, total_errors / len(df) * 100))
print("No Pre-Ov calculated: {}".format(no_calc))
print("Different calculated value: {}".format(diff_calc))
print("Accuracy: {}".format(100 * (1 - total_errors / len(df))))

Total number of cycles: 29298
Total diffs: 6690 (22.83%)
No Pre-Ov calculated: 3152
Different calculated value: 3538
Accuracy: 77.16567683800942


Even with hindsight, this method isn't fantastic. 77% accuracy, after the fact?

In [4]:
no_calc = 0
diff_calc = 0

for idx, row in df.iterrows():
    computed_L_PREOVULATION = None
    for i in range(1, 91):
        six_days = [row['TEMP'+str(i+j)] for j in range(0, 6)]
        three_days = [row['TEMP'+str(i+k)] for k in range(6, 9)]
        if min(three_days) > max(six_days):
            computed_L_PREOVULATION = i + 5
            break
    if computed_L_PREOVULATION is None:
        no_calc += 1
    elif abs(computed_L_PREOVULATION - int(row.L_PREOVULATION)) > 1:
        diff_calc += 1
total_errors = no_calc + diff_calc
print("Total number of cycles: {}".format(len(df)))
print("Total big diffs: {} ({:5.2f}%)".format(total_errors, total_errors / len(df) * 100))
print("No Pre-Ov calculated: {}".format(no_calc))
print("Difference greater than 1: {}".format(diff_calc))
print("Softened accuracy: {}".format(100 * (1 - total_errors / len(df))))

Total number of cycles: 29298
Total big diffs: 5650 (19.28%)
No Pre-Ov calculated: 3152
Difference greater than 1: 2498
Softened accuracy: 80.71540719503038


And it doesn't improve much when we relax the problem, so that it can be one day off. Grrrr.